In [ ]:
# Import necessary libraries
import os
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional, BatchNormalization

# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Load your data
categories_path = '/Users/rishabhtiwari/Desktop/training_dataset/categories.npy'
valid_periods_path = '/Users/rishabhtiwari/Desktop/training_dataset/valid_periods.npy'
training_data_path = '/Users/rishabhtiwari/Desktop/training_dataset/training_data.npy'

categories = np.load(categories_path)
valid_periods = np.load(valid_periods_path)
training_data = np.load(training_data_path)

# Constants for sequence and forecast lengths
seq_length = 128     # predictions based on previous seq_length data entries
forecast_length = 9  # predicting forecast_length time steps into the future

# Data preprocessing
def preprocess_data(data, valid_periods, scaler):
    preprocessed_data = []
    for i, row in enumerate(data):
        valid_data = row[valid_periods[i][0]:valid_periods[i][1]]
        if valid_data.size > 0:
            scaled_data = scaler.fit_transform(valid_data.reshape(-1, 1)).flatten()
            preprocessed_data.append(scaled_data)
    return preprocessed_data

# Convert the time series data into sequences
def to_sequences(data):
    X, y = [], []
    for ts in data:
        for i in range(len(ts) - seq_length - forecast_length + 1):
            X.append(ts[i:(i + seq_length)])
            y.append(ts[(i + seq_length):(i + seq_length + forecast_length)])
    return np.array(X), np.array(y)

# Apply preprocessing
scaler = StandardScaler()
data = preprocess_data(training_data, valid_periods, scaler)
X, y = to_sequences(data)

# Data splitting
val_size = int(len(X) * 0.2)
indices = np.arange(len(X))
np.random.shuffle(indices)
X_train, y_train = X[indices[:-val_size]], y[indices[:-val_size]]
X_val, y_val = X[indices[-val_size:]], y[indices[-val_size:]]

# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], 1))

# Model building
def build_advanced_model(input_shape, rnn_units, dropout_rate, forecast_length, use_gru=True):
    input_layer = Input(shape=input_shape)
    if use_gru:
        x = Bidirectional(GRU(rnn_units, return_sequences=True))(input_layer)
    else:
        x = Bidirectional(LSTM(rnn_units, return_sequences=True))(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    if use_gru:
        x = GRU(rnn_units // 2)(x)
    else:
        x = LSTM(rnn_units // 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(forecast_length)(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
dropout_rate = 0.2
rnn_units = 128
forecast_length = y_train.shape[1]  # Assumes y_train is already shaped as needed

model = build_advanced_model(input_shape, rnn_units, dropout_rate, forecast_length, use_gru=True)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Model training
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=1)

model.save('SubmissionModel')